In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from neuralnet import DNN_pred, Abs_LV, Rel_LV
from utils.evaluations import calc_nondiag_score
from sklearn.metrics import average_precision_score, roc_auc_score
from utils.simulation import simulate_glv

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
abundance = np.load(r'D:\microbial_network\microbial_network_explore\data\simulated\n20_k5_random_random_1.0_1000\y.npy')
abundance = abundance / abundance.sum(axis=1, keepdims=True)
adj = np.load(r'D:\microbial_network\microbial_network_explore\data\simulated\n20_k5_random_random_1.0_1000\adj.npy')
adj_norm = adj / adj.sum(axis=1, keepdims=True)
M = np.load(r'D:\microbial_network\microbial_network_explore\data\simulated\n20_k5_random_random_1.0_1000\M.npy')

In [51]:
z, x, y, adj, M = simulate_glv(
        num_taxa=10,
        # avg_degree=np.random.randint(5, 20),
        avg_degree=2,
        time_points=10000,
        time_step=0.01,
        downsample=1,
        noise_var=0,
        max_interaction_strength=1,
    )

Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:10    0:10
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added


In [52]:
model = DNN_pred(20, 512).to('cuda')

loss_fn = torch.nn.MSELoss().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10000, gamma=0.9)

In [53]:
epoch_num = 300000
for epoch in range(epoch_num):
    input_data = torch.from_numpy(abundance[:-1, :]).float().to('cuda')
    target_data = torch.from_numpy(abundance[1:, :]).float().to('cuda')
    # print(input_data.shape, target_data.shape)
    output = model(input_data)
    loss = loss_fn(output, target_data)

    # identity = torch.eye(output.shape[1]).to('cuda')
    # id_out = model(identity)
    # lamda = 0.001
    # loss = loss + lamda * torch.abs(id_out).sum()
    
    print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[1K\r')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

KeyboardInterrupt: 

In [28]:
input_mat = torch.eye(abundance.shape[1]).to(device)
output = np.abs(model(input_mat).detach().cpu().numpy())
calc_nondiag_score(output, adj, metrics=[average_precision_score, roc_auc_score])

[0.29848434836800364, 0.5378928571428572]

In [68]:
model = Abs_LV(10).to('cuda')
loss_fn = torch.nn.MSELoss().to('cuda')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)

In [69]:
epoch_num = 300000
for epoch in range(epoch_num):
    input_data = torch.from_numpy(z[:-1, :]).float().to('cuda')
    target_data = torch.from_numpy(z[1:, :]).float().to('cuda')
    # print(input_data.shape, target_data.shape)
    output = model(input_data)
    loss = loss_fn(output, target_data)

    sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
    loss += sparse_loss * 1e-10
    
    print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[1K\r')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()

In [70]:
a = model.interaction.data.cpu().numpy()
g = model.g.data.cpu().numpy()

In [71]:
calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])

[0.4266637566996519, 0.5928571428571427]

In [60]:
calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])

[0.38180784990377203, 0.6128571428571429]

In [79]:
%%capture
prauc = []
rocauc = []
for i in range(50):
    z, x, y, adj, M = simulate_glv(
            num_taxa=5,
            # avg_degree=np.random.randint(5, 20),
            avg_degree=2,
            time_points=1000,
            time_step=0.01,
            downsample=1,
            noise_var=0,
            max_interaction_strength=1,
        )
    model = Abs_LV(5).to('cuda')
    loss_fn = torch.nn.MSELoss().to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)
    epoch_num = 300000
    for epoch in range(epoch_num):
        input_data = torch.from_numpy(z[:-1, :]).float().to('cuda')
        target_data = torch.from_numpy(z[1:, :]).float().to('cuda')
        # print(input_data.shape, target_data.shape)
        output = model(input_data)
        loss = loss_fn(output, target_data)

        sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
        loss += sparse_loss * 1e-10
        
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[2K\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
    a = model.interaction.data.cpu().numpy()
    g = model.g.data.cpu().numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)


In [82]:
print(np.mean(prauc))
print(np.mean(rocauc))

0.8090201423103899
0.8448


In [85]:
np.savetxt('../data/results/abslv_prauc.txt', prauc)
np.savetxt('../data/results/abslv_roc.txt', rocauc)

In [2]:
# %%capture
prauc = []
rocauc = []
for i in range(1):
    z, x, y, adj, M = simulate_glv(
                num_taxa=5,
                # avg_degree=np.random.randint(5, 20),
                avg_degree=2,
                time_points=1000,
                time_step=0.01,
                downsample=1,
                noise_var=0,
                max_interaction_strength=1,
            )
    model = Rel_LV(5).to('cuda')
    # loss_fn = torch.nn.MSELoss().to('cuda')
    loss_fn = torch.nn.L1Loss().to('cuda')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.9)
    epoch_num = 300000
    for epoch in range(epoch_num):
        input_data = torch.from_numpy(x[:-1, :]).float().to('cuda') * 10000
        target_data = torch.from_numpy(x[1:, :]).float().to('cuda')
        # print(input_data.shape, target_data.shape)
        output = model(input_data, steps=10)
        loss = loss_fn(output, target_data)

        # sparse_loss = torch.norm(model.interaction, p=1) + torch.norm(model.g, p=1)
        # loss += sparse_loss * 1e-10
        
        print(f"Epoch: {epoch}, Loss: {loss.item()}", end='\x1b[2K\r')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

    a = model.interaction.data.cpu().numpy()
    g = model.g.data.cpu().numpy()
    pr, roc = calc_nondiag_score(np.abs(a), adj, metrics=[average_precision_score, roc_auc_score])
    prauc.append(pr)
    rocauc.append(roc)

Node 'mgx' initialized
Interaction 'mgx->mgx' added
set m:(mgx)->(mgx):   0:5    0:5
Added x0 vector to node mgx
Added growth rates to node mgx
Initialized
Interaction 'mgx_mgx' added


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [1, 5]], which is output 0 of AsStridedBackward0, is at version 10; expected version 9 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [3]:
print(np.mean(prauc))
print(np.mean(rocauc))
# np.savetxt('../data/results/rellv_prauc.txt', prauc)
# np.savetxt('../data/results/rellv_roc.txt', rocauc)

0.5071734852694605
0.32000000000000006


Compare ours and their program